## Vehicle acceleration run - part 1: Ideal torque source

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Ideal torque source and no drag of any kind
This first example considers a car with an ideal constant source of torque, no internal friction, no frction with the road, and no aerodynamic drag. This is a very ideal case

Function torque_tyre calculates the torque at the drive wheels simply by multiplying hte motor torque by the gear ratio

In [ ]:
def torque_tyre(Engine_torque, Gear_ratio):
    Tyre_torque = Engine_torque * Gear_ratio
    return Tyre_torque

Function force_tyre calculates the traction force considering the torque at the wheels and the tyre radius

In [ ]:
def force_tyre(Tyre_torque, Tyre_radius):
    Tyre_force = Tyre_torque / Tyre_radius
    return Tyre_force

Function acceleration_car_simple calculates the acceleration of the car considering only the tractive force

In [ ]:
def acceleration_car_simple(Tyre_force, Vehicle_mass):
    Vehicle_acceleration = Tyre_force / Vehicle_mass
    return Vehicle_acceleration

The parameters we need are the vehicle mass $Vehicle\_mass$ in kg, the torque source $Torque$ in Nm, the gear ratio $Gear\_diff$, and the tyre radius $Tyre\_radius$ in m.

In [ ]:
Vehicle_mass = 1000
Torque = 240
Gear_diff = 3.5
Tyre_radius = 0.32

Now we enter the initial conditions: velocity $V\_0$, position $S\_0$ and time $T\_0$
The target is to calculate the time it takes for the vehicle to reach a final speed from stopped. that final speed is entered as a parameter in km/h, so it needs to be converted to m/s for the model to work properly. Once the calculations are done, the speed time series are logged in km/h

In [ ]:
V_0 = 0
S_0 = 0
T_0 = 0
Stop_condition = False
Throttle = 1
Tyre_circ = Tyre_radius * 2 * np.pi
Timestep = 0.001
Time = T_0
Vehicle_velocity = V_0 / 3.6
Vehicle_position = S_0
Final_speed = 100
Final_speedms = Final_speed / 3.6
Time_log = [T_0]
Speed_log = [V_0]
Acc_log = [0]
Position_log = [S_0]
while Stop_condition == False:
    Tyre_Nm = Torque * Gear_diff
    Tyre_force = Tyre_Nm / Tyre_radius
    Vehicle_acceleration = acceleration_car_simple(Tyre_force, Vehicle_mass)
    Vehicle_position = Vehicle_position + Vehicle_velocity * Timestep
    Vehicle_velocity = Vehicle_velocity + Vehicle_acceleration * Timestep
    Time = Time + Timestep
    
    Time_log.append(Time)
    Position_log.append(Vehicle_position)
    Speed_log.append(Vehicle_velocity*3.6)
    Acc_log.append(Vehicle_acceleration)
    
    
    if (Vehicle_velocity >= Final_speedms) or (Vehicle_acceleration < 0.01):
        Stop_condition = True

In [ ]:
plt.plot(Time_log, Speed_log)
plt.grid(linestyle='--')
plt.xlabel('Time (s)')
plt.ylabel('Speed (km/h)')
plt.title('Acceleration 0-'+ str(Final_speed) + ' km/h')
plt.show()

### Ideal torque source with rolling resistance and aerodynamic drag
This second case still considers an ideal constant torque source but includes rolling resistance, -the friction between the tyres and the road-, and aerodynamic drag 

For this case we need to define some new functions. The first one is the new acceleration of the car, which now takes into account the rolling resistance and the aero drag. The total drive force is the result of the tractive force minus the rolling and aero resistances

In [ ]:
def acceleration_car(Tyre_force, Rolling_resistance, Aero_drag, Vehicle_mass):
    Vehicle_acceleration = (Tyre_force - Rolling_resistance - Aero_drag) / Vehicle_mass
    return Vehicle_acceleration

The aerodynamic drag is calculated with an equation that includes the aerodynamic coefficient, front surface, air density, and the square of the linear speed

In [ ]:
def air_drag(Air_coeff, Front_surface, Air_density, Vehicle_velocity):
    Drag = 0.5 * Air_coeff * Front_surface * Air_density * Vehicle_velocity**2
    return(Drag)

The rolling resistance is considered as independent of the speed, and just as the normal reaction from the road multiplied by a coefficient

In [ ]:
def roll_drag(Vehicle_mass, Roll_coeff):
    Roll_drag = Vehicle_mass * Roll_coeff * 9.81
    return Roll_drag

Some new parameters are required: roling resistance coefficient, aerodynamic coefficient, front surface and air density

In [ ]:
Vehicle_mass = 1000
Roll_coeff = 0.02
Air_coeff = 0.7
Front_surface = 2
Air_density = 1.202
Gear_diff = 3.5
Tyre_radius = 0.32
Tyre_circ = Tyre_radius * 2 * np.pi
Timestep = 0.001

using a similar loop as before but adding the new resistances, we now calculate the tiem to accelerate to a higher speed, to see clearly the effect of the aero drag

In [ ]:
V_0 = 0
S_0 = 0
T_0 = 0
Stop_condition = False
Throttle = 1
Tyre_circ = Tyre_radius * 2 * np.pi
Timestep = 0.001
Time = T_0
Vehicle_velocity = V_0 / 3.6
Vehicle_position = S_0
Final_speed = 180
Final_speedms = Final_speed / 3.6
Time_log = [T_0]
Speed_log = [V_0]
Acc_log = [0]
Position_log = [S_0]
while Stop_condition == False:
    Tyre_Nm = Torque * Gear_diff
    Tyre_force = Tyre_Nm / Tyre_radius
    Roll_drag = roll_drag(Vehicle_mass, Roll_coeff)
    Air_drag = air_drag(Air_coeff, Front_surface, Air_density, Vehicle_velocity)
    Vehicle_acceleration = acceleration_car(Tyre_force, Roll_drag, Air_drag, Vehicle_mass)
    Vehicle_position = Vehicle_position + Vehicle_velocity * Timestep
    Vehicle_velocity = Vehicle_velocity + Vehicle_acceleration * Timestep
    Time = Time + Timestep
    
    Time_log.append(Time)
    Position_log.append(Vehicle_position)
    Speed_log.append(Vehicle_velocity*3.6)
    Acc_log.append(Vehicle_acceleration)
    
    
    if (Vehicle_velocity >= Final_speedms) or (Vehicle_acceleration < 0.01):
        Stop_condition = True

In [ ]:
Slope_0 = (Speed_log[10] - Speed_log[0]) / (Time_log[10] - Time_log[0])
Aux = Time_log[-1] * Slope_0
plt.plot([0,Time_log[-1]], [0, Aux], linestyle = '-.', color='red', linewidth=0.5)
plt.plot(Time_log, Speed_log)
plt.grid(linestyle='--')
plt.xlabel('Time (s)')
plt.ylabel('Speed (km/h)')
plt.title('Acceleration 0-'+ str(Final_speed) + ' km/h')
plt.legend(['Without aero drag', 'With aero drag'])
plt.show()